In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
recordC=pd.read_csv("../input/categories/recordC (1).csv")

In [3]:
recordC.head()

,document_id,status,document_url,media_type,media_link,relative_media_path,is_media_downloaded,transcription_text,transcription_text_relative_path,from_time,to_time,project_id,Category,Unnamed: 13,Unnamed: 14
0,NMAAHC-004567415_00,Complete,https://transcription.si.edu/transcribe/15369/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/15369/image/NMAAHC-004567415_00.jpg,0,and one hundred dollars. I have never heard Ba...,/15369/transcriptions/NMAAHC-004567415_00.txt,\N,\N,15369,Records - Court,NaN,NaN
1,NMAAHC-004567415_00020,Complete,https://transcription.si.edu/transcribe/15369/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/15369/image/NMAAHC-004567415_00020.jpg,1,The Freedmen's Bureau\r\nvs.\r\nB.M. Richardso...,/15369/transcriptions/NMAAHC-004567415_00020.txt,\N,\N,15369,Records - Court,NaN,NaN
2,NMAAHC-004567415_00021,Complete,https://transcription.si.edu/transcribe/15369/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/15369/image/NMAAHC-004567415_00021.jpg,1,The Freedmen's Bureau\r\nvs.\r\nB. M. Richards...,/15369/transcriptions/NMAAHC-004567415_00021.txt,\N,\N,15369,Records - Court,NaN,NaN
3,NMAAHC-004567415_00022,Complete,https://transcription.si.edu/transcribe/15369/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/15369/image/NMAAHC-004567415_00022.jpg,1,"me one hundred dollars, two suits of clothes a...",/15369/transcriptions/NMAAHC-004567415_00022.txt,\N,\N,15369,Records - Court,NaN,NaN
4,NMAAHC-004567415_00023,Complete,https://transcription.si.edu/transcribe/15369/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/15369/image/NMAAHC-004567415_00023.jpg,1,"questions again, and I gave him the same answe...",/15369/transcriptions/NMAAHC-004567415_00023.txt,\N,\N,15369,Records - Court,NaN,NaN


In [4]:
recordC.shape

(6135, 15)

In [5]:
recordC_text=recordC["transcription_text"]

In [6]:
print(recordC_text[0])

and one hundred dollars. I have never heard Bagby complain but have heard him say that he dared Mr Richardson to whip him 

[[signature]] W.M. Henry [[/signature]]

Signed sealed and sworn to before me, March 19th 1866.

[[signature]] H.H. Holton [[/signature]] J. Peace 

Edward Homer being again sworn saith he never saw or knew of Mr Richardson's whipping or correcting said Bagby, and when good reason and prudence demanded chastisement, and then in a proper manner, that he was never bodily injured, or caused to ly up or stop from labor, or eating, from any such correction

[[signature]] Mr Edward Horner [[/signature]] {seal}

Sworn to and subscribed March 19 1866
[[signature]] H.H. Holton [[/signature]]

[[strikethrough]]
"Examines before Court" Mathew Baldwin being again sworn saith that he has heard Bagly speak of his contract with Mr Richardson he expressed himself satisfied

I advised him that it was a good trade and the contract with it, he never complained of his treatment, said

In [7]:
print(recordC_text[10])

Ans.  He did. About those words.

Ques. by Def.  Did you not hear the paper read and agree to go back?

Ans.  Mr. Parker read it to me, and advised me to go back.

Ques. by Def.  Was any force used to compel you to go back?

Ans.  Yes: he said if I din't go back, he would put me in jail and it would cost me $600.

Ques. by Def.  (Producing paper)  If this is not the paper that was read to you?

Ans.  I do not know, as I can neither read nor write. You are asking me too much now.

Ques. by Def.  Did you agree to the paper binding you?

Ans.  I do not recollect of answering.

Ques. by Def.  Yo have no recollection of assenting to it?

Ans. I did not give any answer at all.

Ques. by Def.  Do you know one John A Bailey?

Ans.  I do.

Ques. by Def.  Have you ever told him anything about Richardson not whipping you?

Ans.  No Sir; I never told any living man that Mr. Richardson had never whipped me.

Ques. by Def.  Did Richardson resort to any force to keep you?

Ans.  Don't think he did.



In [8]:
print(recordC_text[100])

Case of Mr. William Barnes of Hilson W.C. charged with gross abuse to an aged woman of color and her Daughter.

Chany the abused states as follows

II that Miss Sally Barnes wife of the accused beat her with her hands

IIII That not satisfied with this the said Miss Barnes beat her with a shingle, that she "Chaney" caught hold of the shingle, when Mr Barnes appeared and said, Jam turn that shingle loose you g_d d_d old b_h or die knock you in the head with this walking stick-whereupon  she "Chaney" let go of the shingle and suffered Miss Barnes to continue beating her.

IIII That while Mr. Barnes and family were at breakfast, she started for the town of Hilson Hilson Co. to report the case to Capt Bullock of the Socae Polies for said be

IIV That she was turned by back by some person unknown to her who claimed to be a Yankee


In [9]:
print(recordC_text[500])

10
[[2 column table]]
Complaint | Disposition

Nov 4 1867 Mr. R. C. Stuart, White, complains that one Jacob Corcoran. (Black) after contracting to work one year for him, has left the Plantation [with?] gathering the crop &c. | Letter sent to Corcoran to appear at this office Thursday Nov 7, 1867 to explain his reasons for leaving Mr. Stuart before the crop was gathered. Jake appeared and the case settled.

Nov 5 1867 Mr. Z. L. Morris (white) complains that Zenas Hunter & Dave Wallace (Freedmen) are not laboring according to contract | Nov 5th 1867 Letter sent to Zenas Hunter & Dave Wallace to appear at his office Thursday Nov 7 1867 to bring their accounts with them &c Dave appeared and the contract was explained to him.

Nov 6 1867 E. Lowengard, white, complains that one Isaac Harry (Black) owes him $240.00 dolls. and will not pay him anything. Said Isaac owes it to Mr. Lowengard for goods &c purchased | Isaac paid Mr. Lowengard $100.00 dolls.

Nov 7 1867 Willoby Sawyer (Freedman) com

In [10]:
print(recordC_text[700])

[[6 columned table]]

|No.|Date|Plaintiff & Defendant|Nature of Complaint|Action Taken|Remarks|

|80|Aug 13th 1868 L.S. 115|Maria Burns "Frd v.s. James Register "W"|Complains that Register owes her daughter $7 00/100 for two months work.|---|To settle or appear at these Hd.Qrs. Saturday 11 o'clock A.M. August 22nd 1868|

81|Aug 17th 1868 L.S. 116|Tom McDougall "Frd v.s. James Calvin "White"|Complains that Calvin owes his son Willis $11 85/100 less cash $2 00/100|---|To settle at once or report at this office Saturday at 1 o'clock P.M. August 24th 1868|

|82|Aug 19 1868 L.S. 116|Randerson Simmons vs. Sophia Hussil white|Complains that Sophia owe his son Arnold $1 18/100 for labor performed|---|To settle immediately or report at this office Aug 22 Saturday at 12 o'clock A.M. 1868|




In [11]:
print(recordC_text[1005])

135  
Case  
Anthony Galloway Esq complains that their crop is not being divide correct between him and Miss Lizzie McMely
Greensboro, N.C. Dec 8th 67.

Lewis Mr C.A.
Guilford County N.C.

Sir
You are hereby appointed, to divide the crop raised on the plantation of Miss Lizzie MMely by Anthony Galloway. In the contract she agrees to furnish two mules (which includes keeping) and as she did not keep them (the mules) furnished, the account of Anthony against her for feeding mules &c must be allowed.
Very Respectfully Yours
Hugo Hillebrandt
Capt V.R.C. Sub
Asst Comr.

[[left margin]]
Mr Lewis
Reports having setted the case satisfactorily and according to orders &c.
Dec 10th 67.
[[/left margin]]

Case
Willis (w) refuses to settle fairly and according to contract about crop.
Greensboro N.C. December 4th 67.

Willis, Mr E.D.
Rockingham Co N.C.

Sir
Wiley Gun Esq. (col) complains that you have not and will not settle with him - according to agreement, this is therefore to inform you that if y

In [12]:
print(recordC_text[1300])

Thompsonville P.O.
Rockingham Co N.C.
Nov. 2. 1867

Capt Hillebrandt
Greensboro N.C.
Dear Sir
In pusuance to an order issued from your office the 25th [[?]]. I proceeded to listen to the arbitration between Oliver Washington and Capt L L B Smith of this county in relation to the building of a mill dam. The arbiters chosen by the contending parties decided that the contract had not been completed with in any particular as relating to the dam. and I concur with them in this decision. and further it is decided that the said Washington (Col) is not entitled to any pay until he completes the job in accordance with the contract. The arbiters acted upon oath. The arbiter chosen by Capt Smith is Jake Waynick, Washington chose Emily Green of Color.

Very respectfully and truly yours.
Martin Clemmons J.P.



In [13]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'
def process(s):
    s= re.sub("([\[]).*?([\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    return s
for i in tqdm(range(len(recordC_text))):
    recordC_text[i] = process(recordC_text[i])
for i in tqdm(range(len(recordC_text))):
    recordC_text[i] = " ".join(recordC_text[i])

100%|██████████| 6135/6135 [00:02<00:00, 2982.85it/s]


In [14]:
recordC_text

0       one hundred dollars . I never heard Bagby comp...
1       The Freedmens Bureau vs. B.M . Richardson Assa...
2       The Freedmens Bureau vs. B. M. Richardson , Co...
3       one hundred dollars , two suits of clothes wou...
4       questions , I gave answer , I whipped.— All I ...
                              ...                        
6130    Salisbury Jan 8th 67 Maj W Swortts Supt West D...
6131    Your obedient Servant Wager Swayne Major Gener...
6132    P of Atty J. W. Suber To Edward Fullings Given...
6133    State of North Carolina Mecklenburg County Thi...
6134    State of North Carolina Bladen County Alfred ....
Name: transcription_text, Length: 6135, dtype: object

In [15]:
recordC_text[700]

'No.DatePlaintiff DefendantNature of ComplaintAction TakenRemarks 80Aug 13th 1868 L.S . 115Maria Burns Frd v.s . James Register WComplains Register owes daughter 7 00100 two months work.To settle appear Hd.Qrs . Saturday 11 oclock A.M. August 22nd 1868 81Aug 17th 1868 L.S . 116Tom McDougall Frd v.s . James Calvin WhiteComplains Calvin owes son Willis 11 85100 less cash 2 00100To settle report office Saturday 1 oclock P.M. August 24th 1868 82Aug 19 1868 L.S . 116Randerson Simmons vs. Sophia Hussil whiteComplains Sophia owe son Arnold 1 18100 labor performedTo settle immediately report office Aug 22 Saturday 12 oclock A.M. 1868'

In [16]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 43.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-p

In [17]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-28 10:03:59,884 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-28 10:04:25,941 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [18]:
from flair.models import SequenceTagger

tagger1 = SequenceTagger.load("flair/ner-english")

Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-06-28 10:04:37,067 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-28 10:04:38,978 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [19]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-28 10:05:32,411 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-28 10:05:49,495 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


# ontonotes

In [20]:
lst=[]
label=[]
Per=[]
Date=[]
GPE=[]
ORG=[]
LOC=[]
Bulding=[]
Cardinal=[]
from flair.data import Sentence

recordC_onto=pd.DataFrame({"recordC_text":"", "Name-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})
for i in range(len(recordC_text)):
        sentencess =Sentence(recordC_text[i])
        recordC_onto.loc[i,"recordC_text"]=recordC_text[i]
        tagger.predict(sentencess)
        dc={}
        P=[]
        D=[]
        G=[]
        L=[]
        O=[]
        B=[]
        C=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.97) & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
                elif dc[j]=="FAC":
                      B.append(j)
                elif dc[j]=="CARDINAL":
                      C.append(j)
        Per.append(P)
        Date.append(D)
        ORG.append(O)
        GPE.append(G)
        LOC.append(L)
        Bulding.append(B)
        Cardinal.append(C)
        
            
recordC_onto["Date-ontonotes"]=Date
recordC_onto["Name-ontonotes"]=Per
recordC_onto["GPE-ontonotes"]=GPE
recordC_onto["ORG-ontonotes"]=ORG 
recordC_onto["LOC-ontonotes"]=LOC 
recordC_onto["Building-ontonotes"]=Bulding
recordC_onto["Cardinal-ontonotes"]=Cardinal

2022-06-28 10:07:50,861 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 10:07:55,626 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 10:08:15,759 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [21]:
recordC_onto["recordC_text"]=recordC_text

In [22]:
recordC_onto.head()

,recordC_text,Name-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,LOC-ontonotes,Building-ontonotes,Cardinal-ontonotes
0,one hundred dollars . I never heard Bagby comp...,"[Bagby, Richardson, W.M . Henry, H.H . Holton,...",[],[],"[March 19th 1866, March 19 1866]",[],[],[one]
1,The Freedmens Bureau vs. B.M . Richardson Assa...,[B.M . Richardson],[Raleigh],"[The Freedmens Bureau, Freedmens Court, Bureau...",[Mon 28 . 1866],[],[],[165.]
2,"The Freedmens Bureau vs. B. M. Richardson , Co...","[B. M. Richardson, Daniel Bagley, Richardson]",[],[The Freedmens Bureau],[twenty one years old],[],[],[twenty one]
3,"one hundred dollars , two suits of clothes wou...",[Genl Hawley],[],[],[twenty one],[],[],[two]
4,"questions , I gave answer , I whipped.— All I ...","[Bagley, Richardson]",[Martain County],[],[],[],[],[]


# Post processing for dates

In [23]:
recordC_onto["Date-ontonotes"]

0                        [March 19th 1866, March 19 1866]
1                                         [Mon 28 . 1866]
2                                  [twenty one years old]
3                                            [twenty one]
4                                                      []
                              ...                        
6130                                            [Jan 8th]
6131                                    [January 29 1867]
6132    [9th day of Feby 1867, 9th day of Feby 1767, T...
6133    [7th day of February AD 1867, 7th day of Feby ...
6134          [18th of last Sept., 22nd day of June 1866]
Name: Date-ontonotes, Length: 6135, dtype: object

In [24]:
G=[]
for i in recordC_onto["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-Z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)
    
            
      

In [25]:
len(GF)

6135

In [26]:
recordC_onto["postDate-ontonotes"]=GF

In [27]:
G=[]
for i in recordC_onto["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        final_date4=re.sub(r'(^[A-Za-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
        if final_date4 != j :
            M.append(final_date4)
    G.append(M)
    

recordC_onto["postDate-ontonotes"]=G

# standardizing dates

In [28]:
NG=[]
for i in G:
    ng=[]
    for j in i:
        
        date=re.findall(r'([0-9]{1,2})-([A-Z][a-z]\S*)-([0-9]{2,4})',j)
        if date !=[]:
            dates=[]
            for k in date:
                for l in k:
                    dates.append(l)
                
                
                if dates[1] in ["August","Aug","augt","AUGUST","Augst","Augt","AUG","Augusta"]:
                    dates[1]= "August"
                elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER",'Sepr','Septh']:
                    dates[1]= "Septembre"
                elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                    dates[1]= "October"
                elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER", "The"]:
                    dates[1]= "November"
                elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                    dates[1]= "December"
                elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                    dates[1]= "January"
                elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY"]:
                    dates[1]= "February"
                elif dates[1] in ["March","Mar"]:
                    dates[1]= "March"
                elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                    dates[1]= "April"
                elif dates[1] in ["May","MAY"]:
                    dates[1]= "May"
                elif dates[1] in ["JUN","Jun","June"]:
                    dates[1]= "June"
                elif dates[1] in ["JULY","July","JUL"]:
                    dates[1]= "July"
                j = dates[0]+"-"+dates[1]+"-"+dates[2]
                ng.append(j)
        else :
                date = re.findall(r'(^[A-Za-z]\S*.*.)-([A-Z][a-z]\S*)-([0-9]{2,4})',j ) 
                dates=[]
                for k in date:
                    for l in k:
                        dates.append(l)
                
                
                    if dates[1] in ["August","Aug","augt","AUGUST","Augst",'Augt',"AUG","Augusta"]:
                        dates[1]= "August"
                    elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER","Sepr","Septh", "This", "This "]:
                        dates[1]= "Septembre"
                    elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                        dates[1]= "October"
                    elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER"]:
                        dates[1]= "November"
                    elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                        dates[1]= "December"
                    elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                        dates[1]= "January"
                    elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY"]:
                        dates[1]= "February"
                    elif dates[1] in ["March","Mar"]:
                        dates[1]= "March"
                    elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                        dates[1]= "April"
                    elif dates[1] in ["May","MAY"]:
                        dates[1]= "May"
                    elif dates[1] in ["JUN","Jun","June"]:
                        dates[1]= "June"
                    elif dates[1] in ["JULY","July","JUL"]:
                        dates[1]= "July"
                    if dates[0]== "fifteenth" :
                        dates[0]="15"
                    elif (dates[0]=="sixteenth") |(dates[0]=="This 16 "):
                        dates[0]="16"
                    elif dates[0]=="Twenty":
                        dates[0]="20" 
                    elif dates[0]=="Twentyseventh":
                        dates[0]="27" 
                    elif (dates[0]=="Twenty Eighth") |(dates[0]=="TwentyEight") |(dates[0]=="TwentyEight ") |(dates[0]=="Twentyeight"):
                        dates[0]="28"
                    elif (dates[0]=="Ninth") |(dates[0]=="Ninth ")|(dates[0]=="Augusta 9th ") |(dates[0]=="ninth") |(dates[0]=="ninth ")|(dates[0]=="Augusta 9th"):
                        dates[0]="9"
                    
                    elif dates[0]=='Seventh':
                        dates[0]="7" 
                    elif (dates[0]=='The') |(dates[0]=="The 9th") :
                        dates[0]="9"
                    elif dates[0]=="the 9" :
                        dates[0]="9"
                    elif (dates[0]=="Eighth ") |(dates[0]=="Eighth") |(dates[0]=="eighth") |(dates[0]=="eighth "):
                        dates[0]="8"
                    elif dates[0]=="Seventh ":
                        dates[0]="7"
                    elif (dates[0]=="Twenty ninth ")| (dates[0]=="Twenty ninth") | (dates[0]=="twenty ninth ")|(dates[0]=="Twenty Ninth ")|(dates[0]=="Twenty Ninth") :
                        dates[0]="29"
                    elif (dates[0]=="Thirty") |(dates[0]=="Thirty ") | (dates[0]=="thirtieth "):
                        dates[0]="30"
                    elif (dates[0]=="Thirty first ") |(dates[0]=="Thirty first"):
                        dates[0]="31"
                  
                    
                    elif dates[0]=="first ":
                        dates[0]="1"
                    elif (dates[0]=="Second ") |(dates[0]=="Second") |(dates[0]=="second")|(dates[0]=="second "):
                        dates[0]="2"
                    elif dates[0]=="fourth ":
                        dates[0]="4"
                    elif dates[0]=="tenth ":
                        dates[0]="10"
                    elif (dates[0]=="Fifth ")|(dates[0]=="Fifth") |(dates[0]=="fifth ")|(dates[0]=="fifth"):
                        dates[0]="5"
                    elif (dates[0]=="This") | (dates[0]=="This 7th ")|(dates[0]=="seventh ") |(dates[0]=="seventh") |(dates[0]=="Seventh "):
                        dates[0]="7"
                    elif (dates[0]=="This 31st") |(dates[0]=="Thirty-first "):
                        dates[0]="31"
                    elif (dates[0]=="Thirtieth") |(dates[0]=="Thirtieth ") |(dates[0]=="Thirty"):
                        dates[0]="30"
                    elif dates[0]=="twenty fifth ":
                        dates[0]="25"
                    elif dates[0]=="Twenty seventh ":
                        dates[0]="27"
                    elif (dates[0]=="Twenty sixth ") |(dates[0]=="Twentysixth ")|(dates[0]=="Twentysixth") |(dates[0]=="twentysixth "):
                        dates[0]="26"
                    elif dates[0]=="third ":
                        dates[0]="3" 
                    elif dates[0]=="third":
                        dates[0]="3"
                    elif dates[0]=="Third ":
                        dates[0]="3"
                    elif dates[0]=="Third":
                        dates[0]="3"
                    
                    elif dates[0]=="Tenth":
                        dates[0]="10" 
                    elif dates[0]=="Augusta":
                        dates[0]="9" 
                    elif (dates[0]=="this_sixth ")|(dates[0]=="sixth ")| (dates[0]=="Sixth") | (dates[0]=="Sixth ") |(dates[0]=="this sixth "):
                        dates[0]="6" 
                        
                    elif dates[0]=="Tenth ":
                        dates[0]="10"
                    elif (dates[0]=="twentieth")|(dates[0]=="This 20th "):
                        dates[0]="20"
                    elif (dates[0]=="twenty fourth ") |(dates[0]=="twenty fourth") |(dates[0]=="Twenty fourth ") |(dates[0]=="Twenty fourth"):
                        dates[0]="24"
                    elif (dates[0]=="fifteenth ") |(dates[0]=="Fifteenth ") |(dates[0]=="Fifteenth"):
                        dates[0]="15"
                    elif dates[0]=="twentieth ":
                        dates[0]="20"
                    elif dates[0]=="Eighteenth ":
                        dates[0]="18"
                    elif dates[0]=="Eighteenth":
                        dates[0]="18"
                    elif (dates[0]=="Nineteenth ") |(dates[0]=="Nineteenth"):
                        dates[0]="19"
                    elif dates[0]=="First ":
                        dates[0]="1"
                    elif dates[0]=="First":
                        dates[0]="1"
                    elif (dates[0]=="Sixteenth ")|(dates[0]=="this 16") |(dates[0]=="Sixteenth"):
                        dates[0]="16"
                   
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif dates[0]=="twenty third":
                        dates[0]="23"
                    elif dates[0]=="Twenty third ":
                        dates[0]="23"
                    elif dates[0]=="Twenty Second ":
                        dates[0]="22"
                    elif dates[0]=="Twenty Second":
                        dates[0]="22"
                    elif dates[0]=="Twenty Eighth ":
                        dates[0]="28"
                    elif dates[0]=="Twentysenventh ":
                        dates[0]="27"
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif (dates[0]=="twenty third") |(dates[0]=="Twentythird")|(dates[0]=="Twentythird "):
                        dates[0]="23"
                    elif (dates[0]=="fourteenth ") | (dates[0]=="fourteenth") | (dates[0]=="Fourteenth") | (dates[0]=="Fourteenth "):
                        dates[0]="14"
                    elif (dates[0]=="Eleventh ") | (dates[0]=="Eleventh") |(dates[0]=="eleventh ") :
                        dates[0]="11"
                    elif (dates[0]=="Thirteenth") | (dates[0]=="Thirteenth ") :
                        dates[0]="13"
                    elif (dates[0]=="seventeenth ") |(dates[0]=="seventeenth") |(dates[0]=="Seventeenth ") :
                        dates[0]="17"
                     
                     
                     
                    
                        
                    j = dates[0]+"-"+dates[1]+"-"+dates[2]
                    ng.append(j)
            
            
    NG.append(ng)   

In [29]:
recordC_onto["standard_date-ontonotes"]=NG

In [30]:
recordC_onto.to_csv("recordC_onto.csv",index=False)

# Ner english large 

In [31]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
Other_name=[]
from flair.data import Sentence

recordC_large=pd.DataFrame({"recordC_text":"", "Name-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in range(len(recordC_text)):
        sentencess =Sentence(recordC_text[i])
        recordC_large.loc[i,"recordC_text"]=recordC_text[i]
        tagger2.predict(sentencess)
        dc={}
        P=[]
        L=[]
        G=[]
        O=[]
        Other=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") :
                      O.append(j)
                elif  (dc[j]=="MISC") :
                      Other.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
        Other_name.append(Other)
            

recordC_large["Name-nerenglarge"]=Per
recordC_large["LOC-nerenglarge"]=LOC
recordC_large["ORG-nerenglarge"]=ORG    
recordC_large["other_namelarge"] =Other_name 

2022-06-28 10:12:56,192 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 10:13:01,364 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 10:13:20,698 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [32]:
recordC_large.head()

,recordC_text,Name-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,other_namelarge
0,one hundred dollars . I never heard Bagby comp...,"[Bagby, Richardson, W.M . Henry, J, Edward Hom...",[],[],[]
1,The Freedmens Bureau vs. B.M . Richardson Assa...,"[B.M, Richardson, Daniel Bagley Freedman, E. W...",[Raleigh],"[Freedmens Bureau, Bureau, Freedmens Court, Mi...",[]
2,"The Freedmens Bureau vs. B. M. Richardson , Co...","[B. M. Richardson, Daniel Bagley, Richardson]",[],[Freedmens Bureau],[]
3,"one hundred dollars , two suits of clothes wou...",[Hawley],[],[],[]
4,"questions , I gave answer , I whipped.— All I ...","[Bagley, Richardson]",[Martain County],[],[]


In [33]:
recordC_large.to_csv("recordC_large.csv",index=False)

# Ner english

In [34]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
Other_name=[]
from flair.data import Sentence

recordC_ner=pd.DataFrame({"recordC_text":"", "PERSON-nerenglish":"","LOC-nerenglish":"","ORG-nerenglish":[]})
for i in range(len(recordC_text)):
        sentencess =Sentence(recordC_text[i])
        recordC_ner.loc[i,"recordC_text"]=recordC_text[i]
        tagger1.predict(sentencess)
        dc={}
        P=[]
        L=[]
        O=[]
        Other=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG")  :
                      O.append(j)
                elif  (dc[j]=="MISC") :
                      Other.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
        Other_name.append(Other)
            

recordC_ner["Name-nerenglish"]=Per
recordC_ner["LOC-nerenglish"]=LOC
recordC_ner["ORG-nerenglish"]=ORG 
recordC_ner["Other_name-nerenglish"]=Other_name 

2022-06-28 10:23:55,853 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 10:24:12,617 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2022-06-28 10:25:26,871 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [35]:
recordC_ner.head()

,recordC_text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish,Name-nerenglish,Other_name-nerenglish
0,one hundred dollars . I never heard Bagby comp...,NaN,[],[],"[Bagby, Henry Signed, Richardsons, Edward Horn...",[]
1,The Freedmens Bureau vs. B.M . Richardson Assa...,NaN,[],[Freedmens Bureau],[],[]
2,"The Freedmens Bureau vs. B. M. Richardson , Co...",NaN,[],[Freedmens Bureau],"[B. M. Richardson, Daniel Bagley, Richardson]",[]
3,"one hundred dollars , two suits of clothes wou...",NaN,[],[],[Genl Hawley],[]
4,"questions , I gave answer , I whipped.— All I ...",NaN,[Martain County],[],[Bagley],[]


In [36]:
recordC_ner.to_csv("recordC_ner.csv",index=False)

In [37]:
recordC_onto["Name-nerenglarge"]=recordC_large["Name-nerenglarge"]
recordC_onto["LOC-nerenglarge"]=recordC_large["LOC-nerenglarge"]
recordC_onto["ORG-nerenglarge"]=recordC_large["ORG-nerenglarge"]
recordC_onto["other_namelarge"]=recordC_large["other_namelarge"]
recordC_onto["PERSON-nerenglish"]=recordC_ner["Name-nerenglish"]
recordC_onto["LOC-nerenglish"]=recordC_ner["LOC-nerenglish"]
recordC_onto["ORG-nerenglish"]=recordC_ner["ORG-nerenglish"]
recordC_onto["Other_name-nerenglish"]=recordC_ner["Other_name-nerenglish"]

In [38]:
to_compare_result_recordC=recordC_onto

In [39]:
to_compare_result_recordC.to_csv("to_compare_result_recordC.csv", index=False)